In [14]:
# -*- coding: utf-8 -*-
"""
Descarga múltiples indicadores del BIE/BISE de INEGI y grafica solo el último año disponible.
Genera CSVs individuales por serie y dos combinados: long (inegi_series_long.csv) y wide (inegi_series_wide.csv).
Además, guarda una gráfica del último año como inegi_ult_anio.png.

Requisitos:
    pip install requests pandas matplotlib
"""

import sys
import json
from json import JSONDecodeError
from typing import Optional, Tuple, Dict, List
import requests
import pandas as pd
import time
import matplotlib.pyplot as plt

# ========= CONFIG =========
# Reemplazaste los indicadores por estos 9 IDs:
INDICATORS: Dict[str, str] = {
    "IND_786478": "786478",
    "IND_786479": "786479",
    "IND_786480": "786480",
    "IND_786481": "786481",
    "IND_786482": "786482",
    "IND_786483": "786483",
    "IND_786466": "786466",
    "IND_786467": "786467",
    "IND_786468": "786468",
}

LANG = "es"
GEO = "00"               # cobertura nacional
LATEST = "false"         # "true" último dato; "false" serie histórica (recomendado para análisis)
SOURCE_PRIMARY = "BIE"   # intenta primero aquí
SOURCE_FALLBACK = "BISE" # reintenta aquí si falla

# Token en duro (según lo pediste):
TOKEN = "38bfc289-5f6b-f47c-ba2d-1895f130bfc0"

SESSION = requests.Session()
SESSION.headers.update({"User-Agent": "master-group-inegi-script/1.0"})

def build_url(indicator_id: str, source: str, resp_type: str="json") -> str:
    base = "https://www.inegi.org.mx/app/api/indicadores/desarrolladores"
    path = f"/jsonxml/INDICATOR/{indicator_id}/{LANG}/{GEO}/{LATEST}/{source}/2.0/{TOKEN}"
    return f"{base}{path}?type={resp_type}"

def fetch_series(indicator_id: str, source: str) -> Tuple[Optional[dict], Optional[dict]]:
    """
    Devuelve (error_info, data_json). Si hay datos válidos, error_info=None y data_json contiene la respuesta.
    """
    url = build_url(indicator_id, source, resp_type="json")
    resp = SESSION.get(url, timeout=30)
    status = resp.status_code
    try:
        data = resp.json()
    except JSONDecodeError:
        return {"status": status, "url": url, "text": resp.text[:800]}, None
    if status == 200 and isinstance(data, dict) and data.get("Series"):
        return None, data
    return {"status": status, "url": url, "payload": data}, None

def to_dataframe(payload: dict, indicator_name: str) -> pd.DataFrame:
    serie = payload["Series"][0]
    obs = serie.get("OBSERVATIONS", [])
    df = pd.DataFrame([{
        "date": o.get("TIME_PERIOD"),
        "value": pd.to_numeric(o.get("OBS_VALUE"), errors="coerce"),
        "obs_status": o.get("OBS_STATUS"),
        "obs_note": o.get("OBS_NOTE"),
        "obs_exception": o.get("OBS_EXCEPTION"),
        "obs_source": o.get("OBS_SOURCE"),
        "cober_geo": o.get("COBER_GEO"),
    } for o in obs])
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["indicator_key"] = indicator_name
    df["indicator_id"] = serie.get("INDICATOR")
    df["freq"] = serie.get("FREQ")
    df["unit"] = serie.get("UNIT")
    df["last_update"] = serie.get("LASTUPDATE")
    df["topic"] = serie.get("TOPIC")
    return df.sort_values("date").reset_index(drop=True)

def download_all(indicators: Dict[str, str]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    frames: List[pd.DataFrame] = []

    for name, ind_id in indicators.items():
        print(f"\n==> Consultando {name} (ID={ind_id}) en {SOURCE_PRIMARY} (LATEST={LATEST})…")
        err, data = fetch_series(ind_id, SOURCE_PRIMARY)

        if data is None:
            print(f"Primer intento sin datos. Reintentando en {SOURCE_FALLBACK}…")
            err2, data = fetch_series(ind_id, SOURCE_FALLBACK)
            if data is None:
                print("❌ No se obtuvo serie en ninguno de los bancos para", name)
                print("Detalle 1:", json.dumps(err, ensure_ascii=False)[:800])
                print("Detalle 2:", json.dumps(err2, ensure_ascii=False)[:800] if err2 else "—")
                continue

        df = to_dataframe(data, name)
        print("✅ Metadatos:",
              {"indicator_id": df["indicator_id"].iat[0],
               "freq": df["freq"].iat[0],
               "unit": df["unit"].iat[0],
               "last_update": df["last_update"].iat[0],
               "topic": df["topic"].iat[0]})
        print(df.head(3))

        # CSV individual
        csv_name = f"inegi_{name}_{ind_id}_{'last' if LATEST=='true' else 'hist'}.csv"
        df.to_csv(csv_name, index=False, encoding="utf-8")
        print(f"💾 CSV guardado: {csv_name}")

        frames.append(df)
        time.sleep(0.35)  # evitar rate limits

    if not frames:
        raise SystemExit("No se descargó ninguna serie. Revisa IDs y token.")

    # Combinar todo (formato largo)
    long_df = pd.concat(frames, ignore_index=True).sort_values(["indicator_key", "date"])

    # Pivot (ancho): cada indicador como columna por 'indicator_key'
    wide_df = (long_df
               .pivot_table(index="date", columns="indicator_key", values="value", aggfunc="last")
               .sort_index())

    # Guardados combinados
    long_df.to_csv("inegi_series_long.csv", index=False, encoding="utf-8")
    wide_df.to_csv("inegi_series_wide.csv", encoding="utf-8")
    print("\n💾 Archivos combinados guardados: inegi_series_long.csv, inegi_series_wide.csv")

    return long_df, wide_df

def plot_last_year(wide_df: pd.DataFrame, outfile: str = "inegi_ult_anio.png") -> None:
    """
    Grafica solo el último año disponible (última fecha menos un año) para todas las columnas con datos.
    Guarda la figura en 'outfile'.
    """
    if wide_df.empty:
        print("No hay datos para graficar.")
        return

    # Asegura índice datetime y elimina columnas completamente vacías
    w = wide_df.copy()
    w = w.dropna(how="all")
    if w.empty:
        print("No hay datos para graficar.")
        return

    last_date = w.index.max()
    start_date = last_date - pd.DateOffset(years=1)

    mask = (w.index > start_date) & (w.index <= last_date)
    last_year_df = w.loc[mask].dropna(how="all")

    if last_year_df.empty:
        print("No hay suficientes datos en el último año para graficar.")
        return

    # Una figura con todas las series (sin especificar colores)
    plt.figure(figsize=(11, 5.5))
    last_year_df.plot()
    plt.title(f"Último año disponible: {start_date.date()} → {last_date.date()}")
    plt.xlabel("Fecha")
    plt.ylabel("Valor")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(outfile, dpi=150)
    plt.close()
    print(f"📈 Gráfica guardada: {outfile}")

def main():
    if not TOKEN:
        print("⚠️ Falta TOKEN del INEGI.", file=sys.stderr)
        sys.exit(1)

    long_df, wide_df = download_all(INDICATORS)
    print("\n== Vista previa (wide) ==")
    print(wide_df.tail(5))

    # Graficar último año
    plot_last_year(wide_df, outfile="inegi_ult_anio.png")

if __name__ == "__main__":
    main()



==> Consultando IND_786478 (ID=786478) en BIE (LATEST=false)…


/tmp/ipython-input-1757597919.py:78: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


✅ Metadatos: {'indicator_id': None, 'freq': '6', 'unit': '330', 'last_update': '10/09/2025 03:19:19 p. m.', 'topic': '101700200010004000200010'}
        date      value obs_status obs_note obs_exception obs_source  \
0 2005-01-01  77.398115                None                      198   
1 2005-02-01  77.137656                None                      198   
2 2005-03-01  68.239712                None                      198   

  cober_geo indicator_key indicator_id freq unit                last_update  \
0        00    IND_786478         None    6  330  10/09/2025 03:19:19 p. m.   
1        00    IND_786478         None    6  330  10/09/2025 03:19:19 p. m.   
2        00    IND_786478         None    6  330  10/09/2025 03:19:19 p. m.   

                      topic  
0  101700200010004000200010  
1  101700200010004000200010  
2  101700200010004000200010  
💾 CSV guardado: inegi_IND_786478_786478_hist.csv

==> Consultando IND_786479 (ID=786479) en BIE (LATEST=false)…
✅ Metadatos: {'in

/tmp/ipython-input-1757597919.py:78: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")



==> Consultando IND_786480 (ID=786480) en BIE (LATEST=false)…
✅ Metadatos: {'indicator_id': None, 'freq': '6', 'unit': '330', 'last_update': '10/09/2025 03:19:19 p. m.', 'topic': '101700200010004000200030'}
        date       value obs_status obs_note obs_exception obs_source  \
0 2005-01-01   92.567673                None                      198   
1 2005-02-01   95.288836                None                      198   
2 2005-03-01  100.918712                None                      198   

  cober_geo indicator_key indicator_id freq unit                last_update  \
0        00    IND_786480         None    6  330  10/09/2025 03:19:19 p. m.   
1        00    IND_786480         None    6  330  10/09/2025 03:19:19 p. m.   
2        00    IND_786480         None    6  330  10/09/2025 03:19:19 p. m.   

                      topic  
0  101700200010004000200030  
1  101700200010004000200030  
2  101700200010004000200030  
💾 CSV guardado: inegi_IND_786480_786480_hist.csv


...

/tmp/ipython-input-1757597919.py:78: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


✅ Metadatos: {'indicator_id': None, 'freq': '6', 'unit': '330', 'last_update': '10/09/2025 03:19:19 p. m.', 'topic': '101700200010002000200020'}
        date      value obs_status obs_note obs_exception obs_source  \
0 2005-01-01  74.661026                None                      198   
1 2005-02-01  78.529652                None                      198   
2 2005-03-01  77.581859                None                      198   

  cober_geo indicator_key indicator_id freq unit                last_update  \
0        00    IND_786467         None    6  330  10/09/2025 03:19:19 p. m.   
1        00    IND_786467         None    6  330  10/09/2025 03:19:19 p. m.   
2        00    IND_786467         None    6  330  10/09/2025 03:19:19 p. m.   

                      topic  
0  101700200010002000200020  
1  101700200010002000200020  
2  101700200010002000200020  
💾 CSV guardado: inegi_IND_786467_786467_hist.csv

==> Consultando IND_786468 (ID=786468) en BIE (LATEST=false)…
✅ Metadatos: {'in

/tmp/ipython-input-1757597919.py:78: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")



💾 Archivos combinados guardados: inegi_series_long.csv, inegi_series_wide.csv

== Vista previa (wide) ==
indicator_key  IND_786466  IND_786467  IND_786468  IND_786478  IND_786479  \
date                                                                        
2024-02-01      97.100361  105.982752  109.147639  114.445340  104.404166   
2024-03-01      96.260588  105.701702  109.807870   94.856389   93.974621   
2024-04-01      97.113479  106.678857  109.849692  123.803716  112.540492   
2025-01-01      94.610227  102.814483  108.671637  110.270592   98.794213   
2025-02-01      97.595267  106.032047  108.644660  118.845663  106.539360   

indicator_key  IND_786480  IND_786481  IND_786482  IND_786483  
date                                                           
2024-02-01      91.226227    1.702536   -4.468137   -6.067374  
2024-03-01      99.070418    2.586557    2.545224   -0.040291  
2024-04-01      90.902354   -1.432642   -5.510057   -4.136679  
2025-01-01      89.592530   12.206

<Figure size 1100x550 with 0 Axes>